In [ ]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config.from_object('otsukare.config.BaseConfig')

db = SQLAlchemy(app)

In [ ]:
from otsukare.models import *

In [ ]:
session.query(
    Expense.date,
    func.sum(Expense.value).label('total')
).join(Expense.cost_center
).filter(CostCenter.id.in_([2, 3])
).group_by(Expense.date
).all()

In [ ]:
from otsukare.analysis import * 

In [ ]:
kana_known = pd.read_sql('SELECT * from "Kana_Known"', db.engine.connect().connection)[["user_id","tokens"]].groupby("user_id").sum()
kana_known.columns = ["Kana"]
words_known = pd.read_sql('SELECT * from "Words_Known"', db.engine.connect().connection)[["user_id","tokens"]].groupby("user_id").sum()
words_known.columns = ["Words"]
sent_known = pd.read_sql('SELECT * from "Needs_Known"', db.engine.connect().connection)[["user_id","tokens"]].groupby("user_id").sum()
sent_known.columns = ["Sentences"]

usernames = pd.read_sql('SELECT * from "Users"', db.engine.connect().connection).set_index("id")[["username"]]
usernames.columns = ["Username"]

usernames.join(kana_known).join(words_known).join(sent_known).fillna(0)

In [ ]:
import pandas as pd

In [ ]:
kana_known = pd.read_sql(Kana_Known.query.filter_by(user_id=1).statement, Kana_Known.query.session.bind)

In [ ]:
kana = pd.read_sql(Kana.query.statement, Kana.query.session.bind)

In [ ]:
kana_stats = pd.merge(kana_known, kana, how='inner', left_on="kana_id", right_on="id")

In [ ]:
kana_stats = kana_stats.set_index("kana_id")

In [ ]:
task_stats = pd.read_sql(Tasks.query.filter(Tasks.user_id==1, 
                                      Tasks.task_id>=1, Tasks.task_id<=5, 
                                      Tasks.status!=-1).statement, Kana.query.session.bind)

In [ ]:
incorrect = task_stats[task_stats["status"]==0].groupby("answer").count()["id"]
incorrect.index = incorrect.index.astype(int)
incorrect.name = "incorrect"
correct = task_stats[task_stats["status"]==1].groupby("answer").count()["id"]
correct.index = correct.index.astype(int)
correct.name = "correct"

In [ ]:
correct

In [ ]:
kana_stats = kana_stats.join(correct).join(incorrect)

In [ ]:
kana_stats.columns

In [ ]:
kana_stats = kana_stats[["kana", "romanji", "correct", "incorrect"]]

In [ ]:
kana_stats

In [ ]:
test = task_stats[(task_stats["status"]==0) & (task_stats["response"]!="-1")]

In [ ]:
most_incorrect_x = pd.crosstab(test["answer"],test["response"])
most_incorrect = most_incorrect_x.idxmax(axis=1)


In [ ]:
most_incorrect.index = most_incorrect.index.astype(int)


In [ ]:
most_incorrect = kana_stats.loc[most_incorrect.index,"kana"]
most_incorrect.name = "most incorrect"

In [ ]:
kana_stats = kana_stats.join(most_incorrect)

In [ ]:
kana_stats